In [18]:
import kaggle

In [19]:
! kaggle datasets init -p /app/_data/models/final/0858AddTr/

Data package template written to: /app/_data/models/final/0858AddTr/dataset-metadata.json


In [18]:
! kaggle datasets create -p /app/_data/models/final/0858AddTr/

Starting upload for file eff4_0858_add_tr_3.h5
100%|█████████████████████████████████████████| 203M/203M [05:15<00:00, 675kB/s]
Upload successful: eff4_0858_add_tr_3.h5 (203MB)
Skipping folder: backup; use '--dir-mode' to upload folders
Starting upload for file eff4_0858_add_tr_4.h5
100%|█████████████████████████████████████████| 203M/203M [05:22<00:00, 660kB/s]
Upload successful: eff4_0858_add_tr_4.h5 (203MB)
Starting upload for file eff4_0858_add_tr_1.h5
100%|█████████████████████████████████████████| 203M/203M [05:19<00:00, 667kB/s]
Upload successful: eff4_0858_add_tr_1.h5 (203MB)
Starting upload for file eff4_0858_add_tr_2.h5
100%|█████████████████████████████████████████| 203M/203M [05:16<00:00, 672kB/s]
Upload successful: eff4_0858_add_tr_2.h5 (203MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Starting upload for file eff4_0858_add_tr_5.h5
100%|█████████████████████████████████████████| 203M/203M [05:12<00:00, 681kB/s]
Upload successful: eff4_0858_add

0.891
MODEL_BB_PATH= '../input/model-bb-1/bond_box_999_200.h5'
MODEL_PATH = '../input/0865fulltrain/'
IMAGE_SIZE = (380, 380)
DF_PART = '../input/df-kf-plant/df_kf.csv'
PATH = "/kaggle/input/plant-pathology-2021-fgvc8/"
TRAIN_IMG_PATH = PATH+'train_images/'
TEST_IMG_PATH = PATH+'test_images/'
BATCH_SIZE = 32
NUM_CLASSES=6
SEED = 1488
- replace ''-'scab'
https://www.kaggle.com/nataliayurasova/plant-pathology0891/edit


In [1]:
! pip install albumentations
import albumentations

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

%matplotlib inline

In [3]:
PATH = "/app/_data/"
BATCH_SIZE = 64
SEED = 42
IMAGE_SIZE = 380
NUM_CLASSES = 6
TRAIN_IMG_PATH = "/app/_data/380_npy/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = [
    "complex",
    "frog_eye_leaf_spot",
    "healthy",
    "powdery_mildew",
    "rust",
    "scab",
]
wrong = [
    "ead085dfac287263.jpg",
    "95276ccd226ad933.jpg",
    "da8770e819d2696d.jpg",
    "cd3a1d64e6806eb5.jpg",
    "ccec54723ff91860.jpg",
]

In [4]:
df_labels = pd.read_csv("../_data/df_csv/labels_21_20.csv", index_col=[0])
df_labels["image"] = df_labels["image"].str.replace(".jpg", ".npy")
df_labels = df_labels.query("image not in @wrong").reset_index(drop=True)

In [5]:
df_labels = df_labels.sample(frac=1, random_state=SEED).reset_index(drop=True)

In [6]:
df_labels

,image,labels,complex,frog_eye_leaf_spot,healthy,powdery_mildew,rust,scab
0,ed5407e803f4a9d5.npy,scab,0,0,0,0,0,1
1,c88ca09ea69597d7.npy,frog_eye_leaf_spot,0,1,0,0,0,0
2,Train_1610.npy,healthy,0,0,1,0,0,0
3,90ad674ed881f8b9.npy,scab,0,0,0,0,0,1
4,d0c09d9e21eb6367.npy,healthy,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
20222,cfaab2997580eea0.npy,frog_eye_leaf_spot,0,1,0,0,0,0
20223,d43871073fa3d217.npy,scab,0,0,0,0,0,1
20224,a99fd5a64e6e2860.npy,healthy,0,0,1,0,0,0
20225,85ca3c8fa0b4dfd0.npy,healthy,0,0,1,0,0,0


In [7]:
df_labels.shape

(20227, 8)

In [8]:
df_wrong = pd.read_csv(
    "/app/sandbox/wrong_predictions/eff4/eff4_0891.csv", index_col=[0]
)

In [9]:
len(df_wrong[df_wrong["labels_pred"] != df_wrong["labels_true"]]["image"].unique())
list_wrong = df_wrong[df_wrong["labels_pred"] != df_wrong["labels_true"]][
    "image"
].unique()

2819

In [10]:
df_wrong = df_labels.query("image in @list_wrong").reset_index(drop=True)

In [11]:
# for 380*380
transform = albumentations.Compose(
    [
        albumentations.CLAHE(p=0.1, clip_limit=(1, 2), tile_grid_size=(8, 8)),
        albumentations.OneOf(
            [
                albumentations.MotionBlur((3, 3)),
                albumentations.MedianBlur(blur_limit=3),
                albumentations.GaussianBlur(blur_limit=(3, 3), sigma_limit=0),
                albumentations.Blur(blur_limit=(3, 3)),
            ],
            p=0.2,
        ),
        albumentations.OneOf(
            [
                albumentations.GaussNoise(var_limit=[10, 50], mean=1),
                albumentations.ISONoise(intensity=(0.1, 1), color_shift=(0.01, 0.05)),
                albumentations.ImageCompression(
                    quality_lower=70, quality_upper=100, compression_type=1
                ),
                albumentations.MultiplicativeNoise(
                    multiplier=(0.95, 1.05), per_channel=True, elementwise=True
                ),
                albumentations.Downscale(
                    scale_min=0.6, scale_max=0.99, interpolation=4
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=(-7, 7),
                    sat_shift_limit=(-10, 10),
                    val_shift_limit=(-10, 10),
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=0.15,
                    contrast_limit=0.2,
                    brightness_by_max=True,
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.OpticalDistortion(
                    distort_limit=0.05,
                    shift_limit=0.05,
                    border_mode=2,
                ),
                albumentations.ElasticTransform(
                    alpha=2.0,
                    sigma=50.0,
                    alpha_affine=10.0,
                    interpolation=0,
                    border_mode=2,
                ),
                albumentations.GridDistortion(
                    num_steps=5, distort_limit=0.3, interpolation=0, border_mode=2
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HorizontalFlip(),
                albumentations.VerticalFlip(),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.Rotate(
                    limit=(-180, 180), interpolation=0, border_mode=2
                ),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=180,
                    interpolation=0,
                    border_mode=2,
                ),
            ],
            p=0.5,
        ),
    ]
)

In [12]:
class Generator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        images_src_dir,
        batch_size,
        target_image_size,
        shuffle=False,
        augment=True,
        crop=False,
        resize=False,
        normalize=False,
    ):
        self.shuffle = shuffle
        self.batch_size = BATCH_SIZE
        self.df = df
        self.images_dir = images_src_dir
        self.target_image_size = (IMAGE_SIZE, IMAGE_SIZE)
        self.augment = augment
        self.crop = crop
        self.resize = resize
        self.normalize = normalize
        # create label index map
        self.labels = self._read_labels()
        self.n_samples = self.df.shape[0]
        self.n_batches = self.n_samples // self.batch_size
        # shuffle data, also repeated after each epoch if needed
        if self.shuffle:
            np.random.shuffle(self.labels)

    def _read_labels(self):
        """
        Returns list images mapping to 1-hot label
        """

        # label indexes
        label_ixs = self.df[feature_columns].values
        image_ixs = self.df["image"].values
        labels = []

        for i in range(len(image_ixs)):
            labels.append([image_ixs[i], label_ixs[i]])
        return labels

    def __len__(self):
        """
        Length in batches
        """
        return self.n_batches

    def __getitem__(self, b_ix):
        """
        Produce batch, by batch index
        """

        assert b_ix < self.n_batches

        b_X = np.zeros(
            (self.batch_size, self.target_image_size[0], self.target_image_size[1], 3),
            dtype=np.uint8,
        )

        b_Y = np.zeros(
            (self.batch_size, self.df[feature_columns].shape[1]),
            dtype=np.uint8,
        )

        for i in range(self.batch_size):
            b_X[i], b_Y[i] = self.get_one(
                i + self.batch_size * b_ix,
            )

        return (b_X, b_Y)

    def get_one(self, one_ix):
        """
        Get single item by absolute index
        """
        id = self.labels[one_ix][0]
        src_file = self.images_dir + id

        # read file
        x = np.load(src_file)
        if self.crop:
            coord = self.df[self.df["image"] == id][
                ["x_min", "y_min", "x_max", "y_max"]
            ].values[0]
            orig_hight = x.shape[0]
            orig_width = x.shape[1]
            x_min = coord[0]
            y_min = coord[1]
            x_max = coord[2]
            y_max = coord[3]
            x = x[
                np.int(y_min * orig_hight) : np.int(y_max * orig_hight),
                np.int(x_min * orig_width) : np.int(x_max * orig_width),
            ]

        y = self.labels[one_ix][1]

        # augment
        if self.augment:
            x = self._augment_image(x)

        # normalize (sample-wise)
        if self.normalize:
            x = x.astype(np.float32)
            x = x - np.mean(x, axis=(0, 1))
            x = x / np.std(x, axis=(0, 1))
        return x.astype(np.uint8), y

    def _augment_image(self, x):
        """
        Randomply augment image
        """

        x = transform(image=x)["image"]
        return x

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.labels)

In [13]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA Quadro RTX 5000, compute capability 7.5


In [14]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB4(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b4_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [15]:
BATCH_SIZE = 32

In [16]:
list_models = [
    "eff4_ns_0865_kf_v2_1.h5",
    "eff4_ns_0865_kf_v2_2.h5",
    "eff4_ns_0865_kf_v2_3.h5",
    "eff4_ns_0865_kf_v2_4.h5",
    "eff4_ns_0865_kf_v2_5.h5",
]

In [17]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_wrong["image"], df_wrong["labels"])
):
    train, valid = df_wrong.loc[train_index], df_wrong.loc[valid_index]
    train = train.sample(frac=3, random_state=SEED, replace=True).reset_index(drop=True)
    print(train.shape, valid.shape)


    model_name = "eff4_0858_add_tr_" + str(i + 1) + ".h5"
    log_dir = "logs_0865_add_tr_" + str(i + 1) + "/"
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_f1_score",
            patience=8,
            restore_best_weights=True,
            verbose=1,
            mode="max",
        ),
        keras.callbacks.ModelCheckpoint(
            "/app/_data/models/final/0858AddTr/" + model_name,
            monitor="val_f1_score",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_f1_score",
            factor=0.8,
            patience=3,
            verbose=1,
            mode="max",
            min_delta=1e-4,
            min_lr=0.00000001,
        ),
        keras.callbacks.TensorBoard(
            log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
        ),
        keras.callbacks.experimental.BackupAndRestore("/app/_data/models/final/0858AddTr/backup/")
    ]

    gen_train = Generator(
        df=train,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=True,
        crop=False,
        resize=False,
    )
    gen_valid = Generator(
        df=valid,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=False,
        crop=False,
        resize=False,
    )
    model = keras.models.load_model('/app/_data/models/eff4_kf/0865fulltrain/'+list_models[i])
    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=50,
        steps_per_epoch=train.shape[0] // BATCH_SIZE,
        validation_steps=valid.shape[0] // BATCH_SIZE,
        verbose=1,
        workers=5,
        callbacks=callbacks,
    )
    tf.keras.backend.clear_session()

(6765, 8) (564, 8)
Epoch 1/50
  1/211 [..............................] - ETA: 0s - loss: 0.5464 - acc: 0.5625 - recall: 0.6531 - precision: 0.9412 - f1_score: 0.7111WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
211/211 [==============================] - ETA: 0s - loss: 0.1857 - acc: 0.6909 - recall: 0.8528 - precision: 0.8755 - f1_score: 0.7049
Epoch 00001: val_f1_score improved from -inf to 0.72467, saving model to /app/_data/models/final/0858AddTr/eff4_0858_add_tr_1.h5
211/211 [==============================] - 149s 708ms/step - loss: 0.1857 - acc: 0.6909 - recall: 0.8528 - precision: 0.8755 - f1_score: 0.7049 - val_loss: 0.1592 - val_acc: 0.7261 - val_recall: 0.8922 - val_precision: 0.8975 - val_f1_score: 0.7247
Epoch 2/50
211/211 [=======================

In [ ]:
tf.config.list_physical_devices("GPU")

In [23]:
os.listdir("/app/_data/models/eff4_kf/0865fulltrain/")

['eff4_ns_0865_kf_v2_5.h5',
 'dataset-metadata.json',
 'eff4_ns_0865_kf_v2_3.h5',
 'eff4_ns_0865_kf_v2_2.h5',
 'eff4_ns_0865_kf_v2_4.h5',
 '.ipynb_checkpoints',
 'eff4_ns_0865_kf_v2_1.h5']

In [ ]:
df_labels

In [34]:
def parse_all(file_path):
    #     img = tf.io.read_file(TRAIN_IMG_PATH + file_path)
    #     img = tf.image.decode_jpeg(img, channels=3)
    img = np.load(TRAIN_IMG_PATH + file_path)
    return img


def predict_new(path, model):
    img = parse_all(path)
    img = tf.expand_dims(img, axis=0)
    pred = model.predict(img)
    return pred_to_labels(pred[0])


def pred_to_labels(pred, thresh=0.5, labels=feature_columns):
    pred = [labels[i] for i in range(len(labels)) if pred[i] > thresh]
    pred = " ".join(pred)
    return pred

1. '/app/_data/models/eff4_kf/0865fulltrain/'  best
2. '/app/_data/models/complex/HandComplex/'

In [40]:
# MODEL_LIST = ['eff4_ns_0865_kf_v2_5.h5',
#  'eff4_ns_0865_kf_v2_3.h5',
#  'eff4_ns_0865_kf_v2_2.h5',
#  'eff4_ns_0865_kf_v2_4.h5',
#  'eff4_ns_0865_kf_v2_1.h5']
# MODEL_LIST = [
#     "eff4_ns_cr_hand_complex_0.h5",
#     "eff4_ns_cr_hand_complex_1.h5",
#     "eff4_ns_cr_hand_complex_2.h5",
#     "eff4_ns_cr_hand_complex_3.h5",
#     "eff4_ns_cr_hand_complex_4.h5",
# ]
MODEL_LIST = [
    "eff4_ns_cr_with_predicted_complex_v2_kf1.h5",
    "eff4_ns_cr_with_predicted_complex_v2_kf2.h5",
    "eff4_ns_cr_with_predicted_complex_v2_kf3.h5",
    "eff4_ns_cr_with_predicted_complex_v2_kf4.h5",
    "eff4_ns_cr_with_predicted_complex_v2_kf5.h5",
]

In [39]:
os.listdir("../_data/models/complex/eff4PredictedComplex1")

['eff4_ns_cr_with_predicted_complex_v2_kf2.h5',
 'dataset-metadata.json',
 'eff4_ns_cr_with_predicted_complex_v2_kf1.h5',
 'eff4_ns_cr_with_predicted_complex_v2_kf4.h5',
 'eff4_ns_cr_with_predicted_complex_v2_kf5.h5',
 '.ipynb_checkpoints',
 'eff4_ns_cr_with_predicted_complex_v2_kf3.h5']

In [42]:
df_pred_complex = pd.DataFrame(columns=["image", "labels"])
for i, model_name in enumerate(MODEL_LIST):
    model = keras.models.load_model(
        "/app/_data/models/complex/eff4PredictedComplex1/" + model_name
    )
    for img_name in tqdm(df_labels["image"].values.tolist()):
        pred = predict_new(img_name, model)
        df_pred_complex = df_pred_complex.append(
            {"image": img_name, "labels": pred, "model": i + 1}, ignore_index=True
        )

    print(df_pred_complex.head())

100%|██████████| 20227/20227 [26:32<00:00, 12.70it/s] 


                  image              labels  model
0  ed5407e803f4a9d5.npy                scab    1.0
1  c88ca09ea69597d7.npy  frog_eye_leaf_spot    1.0
2        Train_1610.npy             healthy    1.0
3  90ad674ed881f8b9.npy                scab    1.0
4  d0c09d9e21eb6367.npy             healthy    1.0


100%|██████████| 20227/20227 [27:00<00:00, 12.48it/s] 


                  image              labels  model
0  ed5407e803f4a9d5.npy                scab    1.0
1  c88ca09ea69597d7.npy  frog_eye_leaf_spot    1.0
2        Train_1610.npy             healthy    1.0
3  90ad674ed881f8b9.npy                scab    1.0
4  d0c09d9e21eb6367.npy             healthy    1.0


100%|██████████| 20227/20227 [27:13<00:00, 12.39it/s] 


                  image              labels  model
0  ed5407e803f4a9d5.npy                scab    1.0
1  c88ca09ea69597d7.npy  frog_eye_leaf_spot    1.0
2        Train_1610.npy             healthy    1.0
3  90ad674ed881f8b9.npy                scab    1.0
4  d0c09d9e21eb6367.npy             healthy    1.0


100%|██████████| 20227/20227 [27:23<00:00, 12.31it/s] 


                  image              labels  model
0  ed5407e803f4a9d5.npy                scab    1.0
1  c88ca09ea69597d7.npy  frog_eye_leaf_spot    1.0
2        Train_1610.npy             healthy    1.0
3  90ad674ed881f8b9.npy                scab    1.0
4  d0c09d9e21eb6367.npy             healthy    1.0


100%|██████████| 20227/20227 [27:46<00:00, 12.14it/s]  

                  image              labels  model
0  ed5407e803f4a9d5.npy                scab    1.0
1  c88ca09ea69597d7.npy  frog_eye_leaf_spot    1.0
2        Train_1610.npy             healthy    1.0
3  90ad674ed881f8b9.npy                scab    1.0
4  d0c09d9e21eb6367.npy             healthy    1.0


In [43]:
df_pred_complex = df_pred_complex.merge(
    df_labels[["image", "labels"]],
    on="image",
    how="left",
    suffixes=("_pred", "_true"),
)

In [44]:
df_pred_complex.to_csv("/app/sandbox/wrong_predictions/eff4/df_pred_complex.csv")

In [37]:
# df_hand_complex.to_csv("/app/sandbox/wrong_predictions/eff4/df_hand_complex.csv")

In [45]:
df_pred_complex[df_pred_complex["labels_pred"] != df_pred_complex["labels_true"]][
    "model"
].value_counts()

5.0    3265
1.0    3251
3.0    3016
4.0    3010
2.0    2959
Name: model, dtype: int64

In [25]:
# df_sub.to_csv("/app/sandbox/wrong_predictions/eff4/eff4_0891.csv")

In [20]:
df_sub = pd.read_csv("/app/sandbox/wrong_predictions/eff4/eff4_0891.csv", index_col=[0])

In [21]:
df_sub

,image,labels_pred,model,labels_true
0,ed5407e803f4a9d5.npy,scab,1.0,scab
1,c88ca09ea69597d7.npy,frog_eye_leaf_spot,1.0,frog_eye_leaf_spot
2,Train_1610.npy,healthy,1.0,healthy
3,90ad674ed881f8b9.npy,scab,1.0,scab
4,d0c09d9e21eb6367.npy,healthy,1.0,healthy
...,...,...,...,...
101130,cfaab2997580eea0.npy,frog_eye_leaf_spot,5.0,frog_eye_leaf_spot
101131,d43871073fa3d217.npy,scab,5.0,scab
101132,a99fd5a64e6e2860.npy,healthy,5.0,healthy
101133,85ca3c8fa0b4dfd0.npy,healthy,5.0,healthy


In [22]:
df_sub[df_sub["labels_pred"] == ""]["labels_true"].value_counts()

Series([], Name: labels_true, dtype: int64)

In [27]:
df_sub[df_sub["labels_pred"] != df_sub["labels_true"]]["labels_true"].value_counts()

scab frog_eye_leaf_spot            3410
complex                            1012
scab frog_eye_leaf_spot complex    1000
frog_eye_leaf_spot complex          825
rust frog_eye_leaf_spot             590
rust complex                        455
powdery_mildew complex              435
scab                                342
rust                                302
frog_eye_leaf_spot                  200
powdery_mildew                      117
healthy                             106
Name: labels_true, dtype: int64

In [26]:
df_sub[df_sub["labels_pred"] != df_sub["labels_true"]]["model"].value_counts()

2.0    1974
3.0    1772
4.0    1758
1.0    1666
5.0    1624
Name: model, dtype: int64

In [23]:
list_imgs = df_sub[df_sub["labels_pred"] != df_sub["labels_true"]]["image"].tolist()

In [24]:
len(list_imgs)

8794

In [25]:
len(set(list_imgs))

2819